In [759]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import csv

In [760]:
def make_sentences(tokens,tags):
    """
    Function converts list of words into sentences with sentences of corresponding tags

    INPUT : Dataframe of tokens, Dataframe of tags

    OUTPUT : Zip of list of sentences, list of tags sentences
    """
    data = tokens.join(tags, on="id", how = "inner", rsuffix = "_tag").drop("id_tag",axis=1)
    sentences = []
    tags_list = []
    temp_tokens = []
    temp_tags = []
    for row in data.itertuples():
        word = row[2]
        tag = row[3]
        if word!='-DOCSTART-' and word!='.':
            temp_tokens.append(word)
            temp_tags.append(tag)
        if word=='.':
            sentences.append(' '.join(temp_tokens) + ' .')
            tags_list.append(' '.join(temp_tags) + ' .')
            temp_tokens = []
            temp_tags = []
    
    return zip(sentences,tags_list)

def load_data(sentence_file, tag_file=None):
    """Loads data from two files: one containing sentences and one containing tags.

    tag_file is optional, so this function can be used to load the test data.

    Suggested to split the data by the document-start symbol.

    """
    tokens = pd.read_csv(sentence_file)
    if tag_file:
        tags = pd.read_csv(tag_file)
    else:
        #dummy tags for test file
        tags = pd.DataFrame()
        tags['id'] = range(len(tokens))
        tags['tag'] = ['NNP']*len(tokens)

    return make_sentences(tokens,tags)

In [761]:
train_data = load_data("../data/train_x.csv", "../data/train_y.csv")

In [762]:
#creating word_count dictionary
word_count = Counter()
for sentence in zip(*train_data)[0]:
    for word in sentence.split(' '):
        word_count[word]+=1

In [763]:
def nGramTagger(n):
    dic = {}
    tags = zip(*train_data)[1]
    for line in tags:
        line = line.split(' ')
        line = ['*']*n + line
        for i in range(n,len(line)):
            if n==1:
                item = line[i]
            else:
                item = tuple(line[i-n:i])
            if item in dic:
                dic[item]+=1
            else:
                dic[item]=1
    return dic

def wordTagger():
    dic = defaultdict(int) 
    for line1,line2 in train_data:
        for word,tag in zip(line1.split(' '),line2.split(' ')):
            dic[(word,tag)]+=1
            if word_count[word]<5:
                dic[('UNK',tag)]+=1
    return dic

unigram = nGramTagger(1)
bigram = nGramTagger(2)
trigram = nGramTagger(3)
fourgram = nGramTagger(4)

wordtag = wordTagger()

In [731]:
def good_turing_smoothing(dic):
    reverse_dic = {}
    k=5
    #smoothing for rare words
    for key,value in dic.items():
        if value<=5:
            reverse_dic[value] = reverse_dic.get(value,0)+1
    
    for key,value in dic.items():
        if value<5:
            dic[key] = (value+1)*reverse_dic[value+1]/reverse_dic[value]
    print reverse_dic
    #handling for unseen words or combinations
    dic['UNK'] = reverse_dic.get(1,1)
    
    return dic
    
unigram = good_turing_smoothing(unigram)
bigram = good_turing_smoothing(bigram)
trigram = good_turing_smoothing(trigram)

{}
{1: 139, 2: 94, 3: 54, 4: 49, 5: 30}
{1: 4354, 2: 1828, 3: 1117, 4: 734, 5: 580}


In [765]:
tag_set = unigram.keys()

In [778]:
def get_q(tag_penult,tag_prev,tag_current):
    k=1
    num = float(trigram.get((tag_penult, tag_prev, tag_current),0.0)) + k
    den = float(bigram.get((tag_penult, tag_prev), 0.0)) + k*len(bigram)
    return num/den

def get_e(word,tag):
    if word not in word_count:
        return float(wordtag[("UNK",tag)])/unigram[tag]
    else:
        return float(wordtag[(word,tag)])/unigram[tag]
  
        
        

In [767]:
def prob_distibution(k):
    prob = []
    tag_list = zip(*unigram.keys())[0]
    for key1 in tag_list :
        for key2 in tag_list:
            if (key1,key2) in bigram:
                num = bigram.get((key1,key2),0.0) + k
            else:
                num=k
            den = unigram.get(key1,0.0) + (k*len(tag_list))
            prob.append(float(num)/den if den!=0 else 0.0)
    print prob.count(0.0)
    plt.plot(sorted(prob,reverse=True))
    plt.show()

In [768]:
def sequence_probability(self, sequence, tags):
        """Computes the probability of a tagged sequence given the emission/transition
        probabilities.
        """
        tag_penult = '*'
        tag_prev = '*'
        prod = 1
        for word, tag in zip(sequence,tags):
            q = self.get_q(tag,tag_prev,tag_penult)
            e = self.get_e(word,tag)
            tag_penult = tag_prev
            tag_prev = tagger
            prod *= q*e

        return prod


In [779]:
def inference(sequence,mode):
        """Tags a sequence with part of speech tags.

        You should implement different kinds of inference (suggested as separate
        methods):

            - greedy decoding
            - decoding with beam search
            - viterbi
        """
        #Method 1: Greedy Decoding
        if mode.lower()=='greedy':
            tag_sequence = []
            tag_penult = '*'
            tag_prev = '*'
            for word in sequence.split():
                scores = []
                for tag in tag_set:
                    scores.append(get_q(tag_penult,tag_prev,tag)*get_e(word,tag))
                final_tag = tag_set[np.argmax(scores)]
                tag_sequence.append(final_tag)
                tag_penult = tag_prev
                tag_prev = final_tag
                
            return tag_sequence
        
        #Method 2: Beam Search
        elif mode.lower()=='beam':
            k=3
            sequence = sequence.split()
            best_sequences = [['*','*']]
            for word in sequence:
                scores = {}
                for item in best_sequences:
                    item = list(item)
                    tag_penult = item[-2]
                    tag_prev = item[-1]
                    for tag in tag_set:
                        score = get_q(tag_penult,tag_prev,tag)*get_e(word,tag)
                        item.append(tag)
                        scores[tuple(item)] = score
                        item.pop()
                topk_scores = sorted(scores.items(), key=lambda x: x[1], reverse = True)
                best_sequences = list(zip(*topk_scores)[0][:k])
            
            best_sequence =  best_sequences[0][2:]
            

            return best_sequence
        
        elif mode.lower()=='viterbi':
            tag_sequence= []
            path = {}
            path['*','*'] = []
            pi_func = defaultdict(float)
            pi_func[(0, "*", '*')] = 0.0

            # v = Tag current, u = Tag previous, w = Tag penult
            sequence = sequence.split(" ")
            n = len(sequence)

            for k in range(1,n+1):
                temp_path = {}
                for u in possible_tags(k-1):
                    for v in possible_tags(k):
                        max_tag = ""
                        max_score = float("-Inf")
                        for w in possible_tags(k - 2):
                            score = pi_func.get((k-1, w, u),float('-Inf'))*get_q(w,u,v)*get_e(sequence[k-1],v)
                            if score > max_score:
                                max_score = score
                                max_tag = w
                        pi_func[(k, u, v)] = max_score
                        temp_path[u,v] = path[max_tag,u] + [v]
                path = temp_path

            prob,umax,vmax = max([(pi_func.get((n,u,v))*get_q(u,v,'.'),u,v) for u in possible_tags(n-1) for v in possible_tags(n)])

            return path[umax,vmax]
        
def possible_tags(k):
    if k in (-1, 0):
        return set('*')
    else:
        return tag_set


In [781]:
evaluate(dev_data,'beam')

300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900


(0.9142146859114689, 0.0)

In [711]:
def save_results(predicted_tags, mode):

    if mode.lower()=='dev':
        data_x = pd.read_csv('../data/dev_x.csv')
    
    elif mode.lower()=='test':
        data_x = pd.read_csv('../data/test_x.csv')
    
    predicitons=[]
    for row in data_x.itertuples():
        if row[2]=='-DOCSTART-':
            predicted_tags.insert(row[1],'O')
        
    df = pd.DataFrame()
    df['id'] = range(len(predicted_tags))
    df['tags'] = predicted_tags
    
    if mode.lower()=='dev':
        df.to_csv('../predictions.csv',index=False)
    elif mode.lowe()=='test':
        df.to_csv('../results/test_y.csv',index=False)


In [ ]:
def evaluate(data,mode):
    individual_score = 0
    sentence_score = 0
    total_word_count = 0
    final_results = []
    idx=0
    for sentence, tag_sequence in data:
        #checking number of sentences processed to gauge runtime
        idx+=1
        if idx%100==0:
            print idx
        tag_sequence = tag_sequence.split(' ')
        result = inference(sentence,mode)
        final_results.extend(result)
        if result==tag_sequence:
            sentence_score+=1
            individual_score+=len(result)
            total_word_count += len(result)
        else:
            for predicted, actual in zip(result,tag_sequence):
                total_word_count+=1
                if predicted==actual:
                    individual_score+=1
    
    save_results(final_results,'dev')
    
    return float(individual_score)/total_word_count, float(sentence_score)/len(data)

        

In [593]:
dev_data = load_data("../data/dev_x.csv", "../data/dev_y.csv")
test_data = load_data("../data/test_x.csv")